In [8]:
import ee
import ee.mapclient

ee.Initialize()
#ee.mapclient.centerMap(-107, 41, 6)

#fc = ee.FeatureCollection([
 #   ee.Feature(
  #      ee.Geometry.Polygon(
   #         [[-109.05, 41], [-109.05, 37], [-102.05, 37], [-102.05, 41]]),
    #    {'name': 'Colorado', 'fill': 1}),
    #ee.Feature(
     #   ee.Geometry.Polygon(
      
    #[[-114.05, 37.0], [-109.05, 37.0], [-109.05, 41.0],
     #        [-111.05, 41.0], [-111.05, 42.0], [-114.05, 42.0]]),
      #  {'name': 'Utah', 'fill': 2})
    #])


#collection = ee.ImageCollection('COPERNICUS/S2')
#roi = ee.Geometry.Point([-5.55, 35.96]);
#s2_filtered_1 = collection.filterDate('2017-05-01', '2017-06-01').filterBounds(roi);
#print('Collection filtered by time and location', s2_filtered_1);

#granual_id_list = ee.Dictionary(
  #s2_filtered_1.aggregate_histogram('GRANULE_ID')).keys();
#print('GRANULE_ID list', granual_id_list.getInfo());
#print(collection.size().getIfo())
#collection.getInfo()
#print(collection.getInfo(1))#ee.mapclient.addToMap(collection.get(1))
#image = ee.Image('CGIAR/SRTM90_V4'
#print(prueba.getInfo())
#ee.mapclient.centerMap(prueba, 6)#longitud, latitud y zoom
#ee.mapclient.addToMap(image, {'min': 0, 'max': 3000})

image = ee.Image('CGIAR/SRTM90_V4')
print(image)
ee.mapclient.addToMap(image, {'min': 0, 'max': 3000})

ee.Image({
  "type": "Invocation", 
  "arguments": {
    "id": "CGIAR/SRTM90_V4"
  }, 
  "functionName": "Image.load"
})


RuntimeError: main thread is not in main loop